In [1]:
# Ho estem executant sense utilitzar la GPU

import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [2]:
# Al ser una DB distribuida les consultes poden ser diferents, depenent del node que respongui abans els resultats seran diferents
# Si demanes totes les dades del dataset no té perquè sempre retornar-les amb el mateix ordre

query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,6.686620,True,18,1,43.0
1,9.360828,True,32,1,41.0
2,8.437091,False,30,1,39.0
3,6.124442,False,24,1,40.0
4,7.125340,False,26,1,41.0


In [3]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9989.000000,10000.000000,10000.000000,9890.000000
mean,7.297602,27.298900,1.034400,38.699798
std,1.291685,6.165838,0.192926,2.539957
min,0.612885,12.000000,1.000000,17.000000
25%,6.624891,22.000000,1.000000,38.000000
50%,7.374463,27.000000,1.000000,39.000000
75%,8.124034,32.000000,1.000000,40.000000
max,12.257702,50.000000,3.000000,47.000000


In [4]:
df['is_male'].value_counts()

True     5150
False    4850
Name: is_male, dtype: int64

In [5]:
df.isnull().sum()

weight_pounds       11
is_male              0
mother_age           0
plurality            0
gestation_weeks    110
dtype: int64

In [7]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [9]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [11]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [13]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

In [15]:
# Donde ponemos el learning rate (el optimizador) = RMSprop() ==> tf.keras.optimizers.RMSprop(learning_rate=0.001)

model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [17]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Train on 6670 samples, validate on 742 samples
Epoch 1/10
6670/6670 [==============================] - 1s 147us/sample - loss: 1.2195 - mae: 0.8616 - mse: 1.2195 - val_loss: 1.3560 - val_mae: 0.9119 - val_mse: 1.3560
Epoch 2/10
6670/6670 [==============================] - 1s 160us/sample - loss: 1.2183 - mae: 0.8653 - mse: 1.2183 - val_loss: 1.1193 - val_mae: 0.8227 - val_mse: 1.1193
Epoch 3/10
6670/6670 [==============================] - 1s 181us/sample - loss: 1.1874 - mae: 0.8533 - mse: 1.1874 - val_loss: 1.1136 - val_mae: 0.8215 - val_mse: 1.1136
Epoch 4/10
6670/6670 [==============================] - 1s 158us/sample - loss: 1.1814 - mae: 0.8495 - mse: 1.1814 - val_loss: 1.3864 - val_mae: 0.9237 - val_mse: 1.3864
Epoch 5/10
6670/6670 [==============================] - 1s 158us/sample - loss: 1.1724 - mae: 0.8474 - mse: 1.1724 - val_loss: 1.1034 - val_mae: 0.8178 - val_mse: 1.1034
Epoch 6/10
6670/6670 [==============================] - 1s 158us/sample - loss: 1.1746 - mae: 0.8468 - 

In [19]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])

In [21]:
_ ,val_mse, _ = model.evaluate(x_test, y_test, verbose=0)
val_mse

0.844209

In [23]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  7.8259573
Actual val:  7.87491199864

Predicted val:  7.298541
Actual val:  8.000575487979999

Predicted val:  8.100183
Actual val:  9.56365292556

Predicted val:  7.614209
Actual val:  7.3083239852999995

Predicted val:  7.392612
Actual val:  6.7902376696

Predicted val:  7.5638123
Actual val:  9.31232594688

Predicted val:  7.067197
Actual val:  6.8122838958

Predicted val:  7.2848787
Actual val:  7.68751907594

Predicted val:  7.6576333
Actual val:  6.4374980503999994

Predicted val:  7.7323885
Actual val:  7.3744626639



In [25]:
wit_data = pd.concat([x_test, y_test], axis=1)

In [27]:
def custom_predict(examples_to_infer):
    preds = model.predict(examples_to_infer)
    return preds

In [28]:
config_builder = (WitConfigBuilder(wit_data[:500].values.tolist(), data.columns.tolist() + ['weight_pounds'])
  .set_custom_predict_fn(custom_predict)
  .set_target_feature('weight_pounds')
  .set_model_type('regression'))
WitWidget(config_builder, height=800)

WitWidget(config={'model_type': 'regression', 'label_vocab': [], 'feature_names': ['is_male', 'mother_age', 'p…